# <center>Deep Generative Models</center>
## <center>Seminar 9</center>

<center>05.11.2024</center>

# Noise Conditioned Scored Network
Что помните?

1. Зашумляем с использованием разных $\sigma_1 < \sigma_2 ... < \sigma_T$
2. Учим модель для разных $\sigma$. Модель знает, для какого $t$ она предсказывает шум
3. Семплируем при помощи *Langevin dynamic* - итеративный процесс, когда по чуть-чуть "скатываемся" в $\pi(x)$
4. В конце процесса зашумления получаем $N(0, \sigma_T \cdot I)$ - чтобы можно было семплировать

<center><img src="attachments/Pasted image 20240508023849.png" width=700 /></center>


<center><img src="attachments/Pasted image 20240508023849.png" width=900 /></center>


**Вопрос:** Если мы умеем хорошо предсказывать шум / "расшумлять", то почему бы сразу не расшумлять?

<center><img src="attachments/Pasted image 20240508023849.png" width=900 /></center>


**Вопрос:** Как учить?

**Как учить?**
<center><img src="attachments/Pasted image 20240430191355.png" width=1000 /></center>

**Вопрос:** Как семплировать?

**Как семплировать?**

<center><img src="attachments/Pasted image 20240508023930.png" width=1000 /></center>

**Вопрос:** Какой критерий остановки?

**Примечание**

Чтобы обусловиться на $\sigma_t$ у score-модели мы учим $s(x_t, t)$. Например: 

In [ ]:
class ConditionedResnetBlock(nn.Module):
    def __init__(self, dim: int, num_embeddings: int) -> None:
        super().__init__()
        # you could experiment with this architecture
        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(dim, dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(dim, dim, kernel_size=1),
        )
        self.dim = dim
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=dim)

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        time_embed = self.embedding(y).view(-1, self.dim, 1, 1)
        return x + self.block(x + time_embed)

## Вспомним теорию
**Как пришли к этой модели?**

1. **Langevin dynamic / Score matching** - не нужно знать $\pi(x)$, нужно знать $\nabla_x \pi(x)$ и можно итеративно семплировать

<center><img src="attachments/Pasted image 20240429131855.png" width=800 /></center>

<center><img src="attachments/Pasted image 20240429131855.png" width=800 /></center>

**Вопрос:** Не знаем $\pi(x)$, умеем семплировать. Что делать? 

2. **Denosing score matching** - зашумим предсказания и будем предсказывать шум, НО $\nabla_{x_{\sigma}}q(x_{\sigma}|x)$ можно будет посчитать!

<center><img src="attachments/Pasted image 20240508024107.png" width=1000 /></center>

**Вопрос:** В чем обман?

<center><img src="attachments/Pasted image 20240508024107.png" width=1000 /></center>

1. Ключевая идея теоремы - разложить матожидание для зашумленных семплов $\mathbf{E}_{q(x_{\sigma})} = \mathbf{E}_{\pi(x)}\mathbf{E}_{q(x_{\sigma}|x)}$
2. **В чем обман?** "локальная" score функция

## Итого

1. **Langevin dynamic / Score matching** - не знаем $\pi(x)$, но знаем $\nabla_x \pi(x)$ $\Rightarrow$ можно семплировать из $\pi(x)$
2. **Denosing score matching** - обучаем модель денойзить, получим "локальные" score функции 
3. **Noice Conditioned Score Network** - учим $s(t, x_t)$ для набора $\sigma_1 < \sigma_2 ... < \sigma_T$
	1. $q(x_1) \sim \pi(x), q(x_T) \sim N(0, \sigma_T \cdot I)$
	2. Семплируем при помощи *Langevin dynamic*